In [ ]:
#unduh gurobipy dan matplotlib_scalebar
%pip install gurobipy
%pip install matplotlib_scalebar

In [ ]:
#import module yang dibutuhkan
import pandas as pd
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from google.colab import files
import matplotlib
import numpy as np
import math
import matplotlib.cbook as cbook
from gurobipy import GRB
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
#upload file ke google colab
upload = files.upload()

In [ ]:
#definisikan data frame
df= pd.read_excel("/content/chargingstation_jakpus.xlsx")

In [ ]:
#Jadikan demand sebagai bilangan bulat
df.Demand= df.Demand.astype(int)

In [ ]:
#Melabeli kelurahan dengan numerik
le= LabelEncoder()
df["Kelurahan_encode"]=le.fit_transform(df.Kelurahan)

In [ ]:
#Definisi demand dengan kelurahannya
imed=df.iloc[0:2243]
buat_demand= dict(imed[["Kelurahan_encode", "Demand"]].to_numpy())

In [ ]:
print(buat_demand
)

In [ ]:
df["price"]= 1 # asumsikan semua biaya pembangunan SPKLU sama
df["Jarak(meter)"]= df["Jarak(meter)"].astype(int) #mwngubah menjadi bilangan bulat

In [ ]:
df.No.unique() #melihat label dari jalan yang berpotensi untuk dibangun SPKLU

array([322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334,
       335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347,
       348, 349, 350, 351, 352, 353, 355, 356, 357, 358, 359, 360, 361,
       362, 363, 364, 365, 366, 464, 465, 472, 473, 480, 495, 496])

In [ ]:
#Konstruksi set covering
buat_cs= {}
count=0
jarak_maks= 2000 #Asumsikan 1 lokasi SPKLU mampu memenuhi 2000 km di sekelilingnya
for i in df.No.unique():
  df_imed=df[(df["No"]==i) & (df["Jarak(meter)"]<=jarak_maks)]
  buat_cs[count]= [set(df_imed["Kelurahan_encode"].tolist()),1]
  count+=1


In [ ]:
buat_cs

In [ ]:
#Parameters
total= 7 #Total stasiun
regions, population = gp.multidict(buat_demand)

sites, coverage, jumlah = gp.multidict(buat_cs)

In [ ]:
print(regions)

In [ ]:
#Memodelkan
m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(jumlah) == total, name="total")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize() 

In [ ]:
#memunculkan titik potensial mana yang sebaiknya dibangun
cs_real_val= list(df.No.unique())
A = []
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
      A.append(cs_real_val[tower])
      print(f" Bangun SPKLU di lokasi {cs_real_val[tower]}.")
print(A)

 Bangun SPKLU di lokasi 322.
 Bangun SPKLU di lokasi 333.
 Bangun SPKLU di lokasi 340.
 Bangun SPKLU di lokasi 348.
 Bangun SPKLU di lokasi 362.
 Bangun SPKLU di lokasi 364.
 Bangun SPKLU di lokasi 472.
[322, 333, 340, 348, 362, 364, 472]


In [ ]:
#coverage

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n SPKLU mampu mencoverage: {coverage} %")


 SPKLU mampu mencoverage: 100.0 %


In [ ]:
#Ambil koordinat jalan potensial SPKLU
B = []
C = []
for i in range(0,len(A)):
  for j in range(0,len(df["No"])):
    if df["No"][j] == A[i]:
      df1 = df.X_Jalan.values[j]
      df2 = df.Y_Jalan.values[j]
      B.append(df1)
      C.append(df2)
      j=j+1
print(B)
print(C)

In [ ]:
#Supaya elemen dalam himpunan B dan C unik
B_ = list(dict.fromkeys(B))
C_ = list(dict.fromkeys(C))
B_

In [ ]:
#Kontruksi gambar peta dengan coveragenya
plt.rcParams["figure.figsize"] = [10.00, 10.00]
plt.rcParams["figure.autolayout"] = True
im = plt.imread("PETA1.jpg")
fig, ax = plt.subplots()
#im = ax.imshow(im, extent=[0, 1188, 0, 1066])
im = ax.imshow(im, extent=[694224, 712020, 9310706, 9321896])

x = B_

y = C_
plt.scatter(x, y, label= "circle", color= "green",marker= ".", s= 67777, alpha = 0.3)
# s= 1.2*10**5


#ax.grid(which='major', color = "black", linewidth = 1)
ax.grid(which='minor', color = "blue" , linestyle=':', linewidth=0.8)
ax.minorticks_on()
scalebar = ScaleBar(7*10**5, location = "upper left", color='#b32400', box_color='yellow', box_alpha=1) # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)
plt.show()